# Project 3 - Article Parsing Toolset

## Team 3 Members

- Matthew Dunbar
- Jeffrei Cher
- Basil James

## Project Problem

To parse article content and extract various data from the article contents

- _**Article Classification into a set of pre-defined categories**_
  (a prediction model)

For Sports Articles:
- _**to identify the sport, and extract game/match scores if present**_
- _(to provide stats summaries if present?)_

## Learning Goal

Develop experience:

- Building and deploying models to leverage in real world applications
- Leveraging custom training of LLMs to provide article analysis.
- Using a tool-based (extensible toolbox) approach to provide multiple analytics features


## Dataset

- https://www.kaggle.com/datasets/fabiochiusano/medium-articles  
[size: 190k+, categories: multiple tags per article, large set, Includes titles, full articles, and URLs]

### Retrieve dataset

In [4]:
! kaggle datasets download -d fabiochiusano/medium-articles -p ./data --unzip

Dataset URL: https://www.kaggle.com/datasets/fabiochiusano/medium-articles
License(s): CC0-1.0
 97%|███████████████████████████████████████▉ | 359M/369M [00:03<00:00, 145MB/s]
100%|█████████████████████████████████████████| 369M/369M [00:03<00:00, 116MB/s]


### Local file

In [6]:
!ls -l ./data/

total 1017916
-rw-r--r-- 1 jupyter jupyter 1042340506 Apr  9 16:11 medium_articles.csv


### Fulfill basic Dataframe dependencies

In [3]:
import os

import pandas as pd
from google.cloud import bigquery

import subprocess
import warnings

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
warnings.filterwarnings("ignore")

import json
from pprint import pprint
import markdown
# import tensorflow as tf

# print(tf.version.VERSION)

### Load dataframe

In [9]:
# Load dataset (change filename accordingly)
df = pd.read_csv('./data/medium_articles.csv')

# Display first few rows
df.head()

,title,text,url,authors,timestamp,tags
0,Mental Note Vol. 24,Photo by Josh Riemer on Unsplash\n\nMerry Chri...,https://medium.com/invisible-illness/mental-no...,['Ryan Fan'],2020-12-26 03:38:10.479000+00:00,"['Mental Health', 'Health', 'Psychology', 'Sci..."
1,Your Brain On Coronavirus,Your Brain On Coronavirus\n\nA guide to the cu...,https://medium.com/age-of-awareness/how-the-pa...,['Simon Spichak'],2020-09-23 22:10:17.126000+00:00,"['Mental Health', 'Coronavirus', 'Science', 'P..."
2,Mind Your Nose,Mind Your Nose\n\nHow smell training can chang...,https://medium.com/neodotlife/mind-your-nose-f...,[],2020-10-10 20:17:37.132000+00:00,"['Biotechnology', 'Neuroscience', 'Brain', 'We..."
3,The 4 Purposes of Dreams,Passionate about the synergy between science a...,https://medium.com/science-for-real/the-4-purp...,['Eshan Samaranayake'],2020-12-21 16:05:19.524000+00:00,"['Health', 'Neuroscience', 'Mental Health', 'P..."
4,Surviving a Rod Through the Head,"You’ve heard of him, haven’t you? Phineas Gage...",https://medium.com/live-your-life-on-purpose/s...,['Rishav Sinha'],2020-02-26 00:01:01.576000+00:00,"['Brain', 'Health', 'Development', 'Psychology..."


### Parse DataFrame to remove unwanted columns

In [10]:
# parse the CSV data
CSV_COLUMNS = [
    "title",
    "text",
    "url",
    "authors",
    "timestamp",
    "tags",
]
LABEL_COLUMN = "text"
DEFAULTS = [["na"], ["na"], ["na"], ["na"], ["na"], ["na"]]
UNWANTED_COLS = ["title", "url", "authors", "timestamp"]

DESIRED_COLUMNS = [col for col in CSV_COLUMNS if col not in UNWANTED_COLS]
df = df[DESIRED_COLUMNS]

# Show full column width and prevent truncation
pd.set_option("display.max_colwidth", None)  # Show full text
pd.set_option("display.expand_frame_repr", False)  # Prevent wrapping

df.head()

text                                                                       tags
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           Photo by Josh Riemer on Unsplash\n\nMerry Christmas and Happy Holidays, everyone!\n\nWe just wanted everyone to know how much we appreciate everyone and how thankful we are for all our readers and writers here. We wouldn’t be anywhere without you, so thank you all for bringing informative, vulnerable, and important pieces that destigmatize mental illness and mental health.\n\nWithout further ado, here are ten of our top stories from last week, all of which were curated:\n\n“Just as the capacity to love and inspire is universal so is the capacity to hate and discourage. Irrespective of gender, race, age or religion none of us are exempt from aggressive proclivities. Those who are narcissistically disordered, and accordingly repress deep seated feelings of inferiority with inflated delusions of grandeur and superiority, are more prone to aggression and violence. They infiltrate our interactions in myriad environments from home, work, school and the cyber world. Hence, bullying does not happen in isolation. Although there is a ringleader she looks to her minions to either sanction her cruelty or look the other way.”\n\n“Even though the circumstances that brought me here were sad and challenging, I’m grateful for how this program has changed my life for the better. I can’t help but imagine what life would be like if everyone learned to accept their powerlessness over other people, prioritize their serenity, and take life one step at a time. We’ll never know, but I’d bet the world would be much happier.”\n\n“The prospect of spending a horrible Christmas, locked in on a psychiatric unit, was one of 

### Check formatting within text column

In [69]:
print(df["text"].iloc[4])  # Display only the 'text' column

You’ve heard of him, haven’t you? Phineas Gage. The railroad worker who survived an explosion that involved an iron rod piercing through his left cheek and out of his brain and skull.

Yeah.

I know.

You’re probably wondering “yeah, alright sweet. What about him?” Well, let’s just say that he was a really popular patient for the field of neuroscience (Cherry, par. 1). And what I found the most interesting about this tragic event was the science of his behavior afterward.

For those of you who don’t know much about Phineas Gage, let me fill you in with the help of my research.

Phineas Gage, 25 years old, was a railroad worker in Vermont. One day, at work, he was using an iron rod to handle explosive gun powder. As he was using the iron rod to handle the gun powder, an explosion suddenly occurred. The iron rod then went through his left cheek and brain. Fortunately, he survived and was able to talk and walk after the accident (Cherry, par. 2–3).

Why did people say that Phineas Gage wa

### Parse Data Sets from DataFrame

In [11]:
from sklearn.model_selection import train_test_split

# Split into training and temp (validation + test) set (70% train, 30% temp)
train, temp = train_test_split(df, test_size=0.3, random_state=42)

# Split the temp set into validation and test set (50% validation, 50% test of the 30%)
validate, test = train_test_split(temp, test_size=0.5, random_state=42)

# Print counts of rows in each set
print(f"Training Set Size: {train.shape[0]}")
print(f"Validation Set Size: {validate.shape[0]}")
print(f"Test Set Size: {test.shape[0]}")

Training Set Size: 134657
Validation Set Size: 28855
Test Set Size: 28856


## Solution

### Approach

LLM-based tool(s) with tool-specific training, with tool-specific engineered prompt(s).
Modular. Function-based.  Extensible.


#### Input: content of the article to analyse

(ideally, the contents directly.  alternate consideration might be to provide a URL, but that would require additional python supporting fucntions to pull, then clean up the contents prior to submission)

#### Output: variable per tool/function

### Instantiate an LLM, and set it up to support one or more functions

#### Ensure the required pip underpinnings are installed (required when having to spin up a new workbench instance)

In [16]:
!pip install google-genai
!pip show google-genai

  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
asl 0.1 requires google-genai==1.2.0, but you have google-genai 1.10.0 which is incompatible.
langchain-google-vertexai 2.0.10 requires httpx<0.28.0,>=0.27.0, but you have httpx 0.28.1 which is incompatible.
Name: google-genai
Version: 1.10.0
Summary: GenAI Python SDK
Home-page: https://github.com/googleapis/python-genai
Author: 
Author-email: Google LLC <googleapis-packages@google.com>
License: Apache-2.0
Location: /opt/conda/lib/python3.10/site-packages
Requires: anyio, google-auth, httpx, pydantic, requests, typing-extensions, websockets
Required-by: asl


#### Instantiate the new LLM

In [39]:
# Instantiate an LLM
from typing import Any, Callable, Optional, Tuple, Union

from google import genai
from google.cloud import bigquery
from google.genai.types import (
    FunctionDeclaration,
    GenerateContentConfig,
    GenerateContentResponse,
    Part,
    Schema,
    Tool,
)
from IPython.display import Markdown

REGION = "us-central1"
PROJECT = !(gcloud config get-value core/project)
PROJECT = PROJECT[0]

MODEL = "gemini-2.0-flash-001"

client = genai.Client(vertexai=True, location="us-central1")

# Define the ChatAgent parent class
class ChatAgent:
    def __init__(
        self,
        tools: list[Tool],
        tool_handler_fn: Callable[[str, dict], Any],
        max_iterative_calls: int = 5,
    ):
        
        self.tools = tools
        self.tool_handler_fn = tool_handler_fn
        
        # Define the generate_content_config
        generate_content_config = GenerateContentConfig(
            max_output_tokens=3,
            response_modalities=["TEXT"],
            safety_settings=[
                SafetySetting(
                    category="HARM_CATEGORY_HATE_SPEECH",
                    threshold="OFF"
                ),
                SafetySetting(
                    category="HARM_CATEGORY_DANGEROUS_CONTENT",
                    threshold="OFF"
                ),
                SafetySetting(
                    category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
                    threshold="OFF"
                ),
                SafetySetting(
                    category="HARM_CATEGORY_HARASSMENT",
                    threshold="OFF"
                ),
            ]
        )        

        self.chat_session = chat = client.chats.create(
            model=MODEL,
            config=generate_content_config,
        )
        self.max_iterative_calls = 5

    def send_message(self, message: str) -> GenerateContentResponse:
        response = self.chat_session.send_message(message)
        # This is None if a function call was not triggered
        fn_calls = response.function_calls

        num_calls = 0
        # Reasoning loop. If fn_calls is empty then we never enter this
        # and simply return the response
        while fn_calls:
            if num_calls > self.max_iterative_calls:
                break

            # Handle the function calls
            fn_call_responses = []
            for fn_call in fn_calls:
                response = self.tool_handler_fn(
                    fn_call.name, dict(fn_call.args)
                )
                fn_call_responses.append(
                    Part.from_function_response(
                        name=fn_call.name,
                        response={
                            "content": response,
                        },
                    ),
                )
                num_calls += 1

            # Send the function call result back to the model
            response = self.chat_session.send_message(fn_call_responses)

            # If the response is another function call then we want to
            # stay in the reasoning loop and keep calling functions.
            fn_calls = response.function_calls

        return response

### Tools

#### Article Topic Classifier

input: article body  
output: one of a curated set of topic categories, based upon highest probability match

##### Create Custom Query and Response Templates

###### LLM Input

In [57]:
# Define the function declaration for the topic classifier
from pydantic import BaseModel, Extra, Field
from typing import Dict, Any
from enum import Enum

import logging
import traceback

# Custom JSON Formatter for logging
class JSONFormatter(logging.Formatter):
    def format(self, record):
        log_obj = {
            "timestamp": self.formatTime(record, self.datefmt),
            "level": record.levelname,
            "message": record.getMessage(),
            "article_text": getattr(record, 'article_text', 'No article text provided'),
            "row_index": getattr(record, 'row_index', 'No row index provided'),
            "stack_trace": getattr(record, 'stack_trace', traceback.format_exc() if record.exc_info else 'No stack trace available')
        }
        return json.dumps(log_obj)

# Get the root logger
logger = logging.getLogger()
logger.setLevel(logging.ERROR)

# Clear existing handlers (important in Jupyter)
logger.handlers.clear()

# Create new handlers
file_handler = logging.FileHandler("./data/error_log.json", mode='a')
stream_handler = logging.StreamHandler()

# Assign formatter
formatter = JSONFormatter()
file_handler.setFormatter(formatter)
stream_handler.setFormatter(formatter)

# Add handlers
logger.addHandler(file_handler)
logger.addHandler(stream_handler)

    
# Define an Enum for topics
class TopicEnum(str, Enum):
    arts = "arts"
    business = "business"
    entertainment = "entertainment"
    culture = "culture"
    literature = "literature"
    medicine = "medicine"
    music = "music"
    personal_development = "personal development"
    philosophy = "philosophy"
    politics = "politics"
    religion = "religion"
    science = "science"
    sports = "sports"
    technology = "technology"
    us_news = "us news"
    world = "world"

    # Use a @classmethod to safely store descriptions
    @classmethod
    def _get_descriptions(cls):
        return {
            "arts": "Cultural and creative activities, including fine arts, theater, and music.",
            "business": "The activities related to the production, distribution, and sale of goods and services.",
            "entertainment": "Media, performance, and activities designed to entertain an audience.",
            "culture": "The shared customs, arts, and social institutions of a particular group of people.",
            "literature": "Written works, books, essays, poems, et cetera, especially those considered of superior or lasting artistic merit.",
            "medicine": "The field of health and healing, including clinical practices and healthcare.",
            "music": "An art form that uses sound and rhythm to express emotions, ideas, and cultural identity.",
            "personal_development": "Activities and practices that improve awareness and identity, develop talents, and enhance the quality of life.",
            "philosophy": "The study of fundamental questions regarding existence, knowledge, ethics, reason, and the mind.",
            "politics": "The activities associated with governance, policy, and political ideologies.",
            "religion": "The system of beliefs, practices, and worship regarding a deity or deities.",
            "science": "Systematic enterprise that builds and organizes knowledge through testable explanations and predictions.",
            "sports": "Physical activities involving skill, competition, and fitness.",
            "technology": "The application of scientific knowledge for practical purposes, particularly in industry.",
            "us_news": "News related to events, politics, and issues within the United States.",
            "world": "Global news, issues, and events occurring internationally."
        }

    @classmethod
    def get_description(cls, topic: "TopicEnum") -> str:
        """Fetches the description for a given topic using the class-level _descriptions dictionary."""
        descriptions = cls._get_descriptions()  # Safely fetch the descriptions
        # Convert spaces in topic.value to underscores to match the dictionary keys
        key = topic.value.replace(" ", "_")
        if key not in descriptions:
            raise ValueError(f"No description found for topic: {topic.value}")
        return descriptions[key]

class Schema(BaseModel):
    type: str
    properties: Dict[str, Any]
    required: list

class FunctionDeclarationWithExtra(BaseModel):
    name: str
    description: str
    parameters: Schema
    # Allow extra fields (like PROMPT)
    class Config:
        extra = Extra.allow
        
topic_list = "\n".join([f"{topic.value.capitalize()}: {TopicEnum.get_description(topic)}" for topic in TopicEnum])

# Define the function declaration for the topic classifier
topic_classifier_tool_handler_fn = FunctionDeclarationWithExtra(
    name="topic_classifier",
    description="Identify the article topic by analysing the contents of the article",
    parameters=Schema(
        type="OBJECT",
        properties={
            "article_text": {  # Define the expected 'article_text' input
                "type": "STRING",
                "description": "The content of the article to classify."
            },
            "topic": {
                "type": "STRING",
                "description": "Topic",
                "enum": [topic.value for topic in TopicEnum]
            },
        },
        required=["article_text", "topic"],
    ),
    PROMPT="""
    Identify the most relevant topic for the following article:

    {article_text}
    
    Review the enumerated topic categories and their descriptions categoried based upon both the topics and descriptions.
    If an areticle IS a poem, the topic is 'literature'.
    Only return one of the enumerated topics:
    
    {topic_list}
    
    """
)

# Function to classify article text based on topic classification
def classify_article_and_get_tags(article_text: str, row_index: int = None):
    try:
        # Dynamically set the prompt with the actual article text
        formatted_prompt = topic_classifier_tool_handler_fn.PROMPT.format(
            article_text=article_text,
            topic_list=topic_list
        )
        
        # Send the article text to the model for classification
        response = client.models.generate_content(
            model=MODEL,
            contents=formatted_prompt,
            config=GenerateContentConfig(
                response_mime_type="text/x.enum",
                response_schema={
                    "type": "STRING",
                    "enum": [topic.value for topic in TopicEnum],
                },
            ),
        )
        
        # The response contains the classification result
        if response is None:
            logger.error(f"Received None response from model for row {row_index}")
            
                
        # Check if the response is valid
        if response and hasattr(response, 'text') and isinstance(response.text, str):
            classification_result = response.text.strip()  # Clean up the result
        else:
            classification_result = "unclassified"  # fallback response

            # Log the issue if response is None
            logger.error(
                "Received None response from model",
                extra={
                    'article_text': article_text,
                    'row_index': row_index,
                    'stack_trace': traceback.format_exc()
                }
            )
        
        return classification_result
    
    except Exception as e:
        # Log any unexpected errors with the row data
        logger.error(
            f"An error occurred while classifying article at row {row_index}: {str(e)}",
            extra={
                'row_index': row_index,
                'stack_trace': traceback.format_exc()
            }
        )
        return "unclassified"  # Fallback

# Now, assuming we have a dataframe or list of articles to process
article_data = [
    "You’ve heard of him, haven’t you? Phineas Gage. The railroad worker who survived an explosion...",
    "Another article text that talks about politics and economy...",
    # Add more articles as needed
]

## Generate the list
# print(topic_list)
# print

# Process each article and classify its topic
for idx, row in test.head(10).iterrows():
    article_text = row["text"]
    predicted_topic = classify_article_and_get_tags(article_text, row_index=idx)
    actual_tags = row["tags"]  # Get the tags for the current row
    print(f"{article_text}\n")
    print("---------------------------------------------------------------------------------\n")
    print(f"Predicted Article Topic: {predicted_topic}\t\tDataset tags: {actual_tags}\n")
    print("---------------------------------------------------------------------------------\n\n")

Lloyd Austin, Secretary of Defense

While Gen. Lloyd Austin would be the first Black Defense Secretary, his terrible record should overshadow this feat, but we know that is going to be brought up in mainstream media. As Gen. Austin was tapped by Obama to begin the process of the withdrawal of American troops from Iraq in 2010, it was an unorganized effort leaving a depleted Iraqi government vulnerable with a military not trained or equipped to quell the rise of ISIS. The disaster did not stop there as Gen. Austin was then given the responsibility of overseeing a Syrian rebel program to combat ISIS that cost us $384 million, which ended in failure after the millions of dollars earmarked to build the training camps were rarely if ever used.

Shortly after all of these debacles, Gen. Austin sold out to defense contractors and wealthy investment funds becoming a board member at Raytheon and a partner at Pine Island Capital Partners, who both stand to profit immensely from a Biden administr

### Having decided that Gemini's accuracy is Surprisingly High

Now create a new dataframe with articles ('text') column, and a new topic column, as a new functional data set.  
(Save every thousand articles to a CSV file, so as not to overload our memory and cause an abend.)

In [55]:
from datetime import datetime

def print_with_timestamp(message: str):
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')  # Format the time
    print(f"{timestamp} - {message}")

new_data_file = "./data/medium.article.topic.csv"
chunk_size = 100

# Start with a clean slate, so we don't make a huge mess within the new csv file while validating
# !rm {new_data_file} 2> /dev/null  # (commented out again for safety purposes)

# Initialize header flag to ensure header is written once
header_written = False

# Initialize a list to store the rows of text and topic
rows = []

# Print dataset length to check if it's large enough
print_with_timestamp(f"Total rows in dataset: {len(df)}")

# Process the DataFrame in chunks
for start_row in range(0, len(df), chunk_size):  # originally started at zero, continuing at 13900 after adding some safety logic following an error interrupting processing
    chunk_df = df.iloc[start_row:start_row + chunk_size]
    print_with_timestamp(f"Processing chunk starting at row {start_row}, ending at row {start_row + chunk_size}")
    
    # Generate the 'topic' column by applying the LLM function to the 'text' column
    chunk_df['topic'] = chunk_df.apply(
        lambda row: classify_article_and_get_tags(
            article_text=row['text'],         # Pass the 'text' column of the row
            row_index=row.name,               # Pass the index (row.name) as row_index
        ), axis=1
    )
    
    # Create a new DataFrame with the 'text' and 'topic' columns
    new_chunk_df = chunk_df[['text', 'topic']]
    
    # Append the chunk to the CSV file, ensuring header is written only once
    new_chunk_df.to_csv(new_data_file, mode='a', index=False, header=not header_written)
    
    # Ensure the header is not written again in subsequent chunks
    header_written = True
    
    # Print after writing
    print_with_timestamp(f"Written rows {start_row} to {start_row + chunk_size} to {new_data_file}")
    print(f"Current file size: {os.path.getsize(new_data_file)} bytes")

print_with_timestamp("Processing complete.")

2025-04-09 21:18:31 - Total rows in dataset: 192368
2025-04-09 21:18:31 - Processing chunk starting at row 0, ending at row 100
2025-04-09 21:19:04 - Written rows 0 to 100 to ./data/medium.article.topic.csv
Current file size: 508612 bytes
2025-04-09 21:19:04 - Processing chunk starting at row 100, ending at row 200
2025-04-09 21:19:37 - Written rows 100 to 200 to ./data/medium.article.topic.csv
Current file size: 1055060 bytes
2025-04-09 21:19:37 - Processing chunk starting at row 200, ending at row 300
2025-04-09 21:20:08 - Written rows 200 to 300 to ./data/medium.article.topic.csv
Current file size: 1633045 bytes
2025-04-09 21:20:08 - Processing chunk starting at row 300, ending at row 400
2025-04-09 21:20:38 - Written rows 300 to 400 to ./data/medium.article.topic.csv
Current file size: 2181381 bytes
2025-04-09 21:20:38 - Processing chunk starting at row 400, ending at row 500
2025-04-09 21:21:10 - Written rows 400 to 500 to ./data/medium.article.topic.csv
Current file size: 2773814

{"timestamp": "2025-04-09 22:29:59,234", "level": "ERROR", "message": "Received None response from model", "article_text": "How to Have a Sex Life When You Have Kids\n\nBecoming a parent doesn\u2019t have to extinguish the flame\n\nPhoto by rawpixel.com from Pexels\n\nI was lucky enough never to have had the misfortune to walk in on my parents during sex. I do have one specific memory of a time when I was about 16 and realized that my parents had just done the deed. As kids, thinking about our parents getting it on was enough to make us run wide-eyed from the room. Now that I\u2019m the parent, I realize how much having children can complicate having an active sex life.\n\nMy kids are 9 and 13, so there\u2019s no more nookie when the kids are napping. Having a fulfilling sex life when you\u2019ve got a family to manage is challenging. With one or both parents often exhausted at day\u2019s end, it\u2019s common to let too much time slip by without finding moments to really connect. Over

2025-04-09 22:30:21 - Written rows 13900 to 14000 to ./data/medium.article.topic.csv
Current file size: 79351815 bytes
2025-04-09 22:30:21 - Processing chunk starting at row 14000, ending at row 14100
2025-04-09 22:30:51 - Written rows 14000 to 14100 to ./data/medium.article.topic.csv
Current file size: 79889789 bytes
2025-04-09 22:30:51 - Processing chunk starting at row 14100, ending at row 14200
2025-04-09 22:31:20 - Written rows 14100 to 14200 to ./data/medium.article.topic.csv
Current file size: 80465177 bytes
2025-04-09 22:31:20 - Processing chunk starting at row 14200, ending at row 14300
2025-04-09 22:32:02 - Written rows 14200 to 14300 to ./data/medium.article.topic.csv
Current file size: 81018920 bytes
2025-04-09 22:32:02 - Processing chunk starting at row 14300, ending at row 14400
2025-04-09 22:32:32 - Written rows 14300 to 14400 to ./data/medium.article.topic.csv
Current file size: 81597640 bytes
2025-04-09 22:32:32 - Processing chunk starting at row 14400, ending at row 1

{"timestamp": "2025-04-09 22:38:27,147", "level": "ERROR", "message": "Received None response from model", "article_text": "Josef Fritzl\n\nWhen Josef Fritzl applied for a permit to expand his basement in 1978, no one thought anything of it. Josef, a successful electrical engineer, was well known in the small Austrian village of Amstetten, where he was born and grew up. Everyone thought of him as an affable family man \u2014 he and his wife, Rosemarie, had seven children.\n\nWhile some might have questioned how strict he was with his wife and children, the prevailing sentiment was, \u201cif it\u2019s not your business, stay out of it.\u201d Only those close to the Fritzl family knew what a tyrant he truly was. He ruled the household with an iron fist, yelling at and beating the children for the slightest infraction. Rosemarie, too, was treated as one of Josef\u2019s subjects, never allowed to have a say in anything, never given any money. If she didn\u2019t obey his every command, she 

2025-04-09 22:38:54 - Written rows 15600 to 15700 to ./data/medium.article.topic.csv
Current file size: 88430071 bytes
2025-04-09 22:38:54 - Processing chunk starting at row 15700, ending at row 15800
2025-04-09 22:39:24 - Written rows 15700 to 15800 to ./data/medium.article.topic.csv
Current file size: 88900318 bytes
2025-04-09 22:39:24 - Processing chunk starting at row 15800, ending at row 15900
2025-04-09 22:39:53 - Written rows 15800 to 15900 to ./data/medium.article.topic.csv
Current file size: 89389701 bytes
2025-04-09 22:39:53 - Processing chunk starting at row 15900, ending at row 16000
2025-04-09 22:40:23 - Written rows 15900 to 16000 to ./data/medium.article.topic.csv
Current file size: 89801047 bytes
2025-04-09 22:40:23 - Processing chunk starting at row 16000, ending at row 16100
2025-04-09 22:40:51 - Written rows 16000 to 16100 to ./data/medium.article.topic.csv
Current file size: 90331819 bytes
2025-04-09 22:40:51 - Processing chunk starting at row 16100, ending at row 1

...

2025-04-10 08:19:25 - Written rows 133900 to 134000 to ./data/medium.article.topic.csv
Current file size: 671111656 bytes
2025-04-10 08:19:25 - Processing chunk starting at row 134000, ending at row 134100
2025-04-10 08:19:56 - Written rows 134000 to 134100 to ./data/medium.article.topic.csv
Current file size: 671491741 bytes
2025-04-10 08:19:56 - Processing chunk starting at row 134100, ending at row 134200
2025-04-10 08:20:26 - Written rows 134100 to 134200 to ./data/medium.article.topic.csv
Current file size: 671707751 bytes
2025-04-10 08:20:26 - Processing chunk starting at row 134200, ending at row 134300
2025-04-10 08:20:55 - Written rows 134200 to 134300 to ./data/medium.article.topic.csv
Current file size: 671980763 bytes
2025-04-10 08:20:55 - Processing chunk starting at row 134300, ending at row 134400
2025-04-10 08:21:25 - Written rows 134300 to 134400 to ./data/medium.article.topic.csv
Current file size: 672481031 bytes
2025-04-10 08:21:25 - Processing chunk starting at row

{"timestamp": "2025-04-10 09:11:24,941", "level": "ERROR", "message": "Received None response from model", "article_text": "Top 5 Tourist Tips\n\nAs a tourist, traveling through Thailand is infinitely better when a Thai girl accompanies you. While many men wisely choose to book ahead and line up many Thai women using online dating, other guys land and then start wondering what to do next.\n\nIs that bewildered guy now you?\n\nHow do you go about quickly find Thai women in Thailand as a newly landed tourist?\n\nMeeting Thai girls when traveling in Thailand is not easy. Opportunities dramatically increase when a traveler uses proven strategies. Our research shows using business cards, asking direct questions, and giving Thai women compliments will provide you with a 260% uplift in meeting opportunities.\n\nWe have got you covered though.\n\nAll of these techniques will work for new tourists. You\u2019ll be entering Thailand through a major airport and a capital city.\n\nLarge cities are 

2025-04-10 09:11:44 - Written rows 144200 to 144300 to ./data/medium.article.topic.csv
Current file size: 723369416 bytes
2025-04-10 09:11:44 - Processing chunk starting at row 144300, ending at row 144400
2025-04-10 09:12:15 - Written rows 144300 to 144400 to ./data/medium.article.topic.csv
Current file size: 723894413 bytes
2025-04-10 09:12:15 - Processing chunk starting at row 144400, ending at row 144500
2025-04-10 09:12:46 - Written rows 144400 to 144500 to ./data/medium.article.topic.csv
Current file size: 724402605 bytes
2025-04-10 09:12:46 - Processing chunk starting at row 144500, ending at row 144600
2025-04-10 09:13:18 - Written rows 144500 to 144600 to ./data/medium.article.topic.csv
Current file size: 724868127 bytes
2025-04-10 09:13:18 - Processing chunk starting at row 144600, ending at row 144700
2025-04-10 09:13:49 - Written rows 144600 to 144700 to ./data/medium.article.topic.csv
Current file size: 725320661 bytes
2025-04-10 09:13:49 - Processing chunk starting at row

{"timestamp": "2025-04-10 12:30:12,916", "level": "ERROR", "message": "Received None response from model", "article_text": "I became aware of the human trafficking for the sex exploitation issue in Italy for the first time in 2015 when I moved to Bologna.\n\nI rented an apartment close to Porta San Vitale with a balcony that looked over an alley where three Nigerian women stayed to meet their clients every night. On the other side of the main road, two Romanian women waited for clients to stop by with their car.\n\nThey were from 19 up to 40 years old. These women are a constant nightly presence on the ring road that surrounds the historic center of Bologna. They are like shadows that appear at dusk and disappear at dawn.\n\nI owe their acquaintance to my dog\u2019s friendliness. She was a rescued dog. I had adopted her from an animal shelter and found out immediately after she was hopelessly sick. Yet, she outlived what the veterinarians initially predicted, and for sure, it was thank

2025-04-10 12:30:17 - Written rows 182200 to 182300 to ./data/medium.article.topic.csv
Current file size: 944345414 bytes
2025-04-10 12:30:17 - Processing chunk starting at row 182300, ending at row 182400
2025-04-10 12:30:49 - Written rows 182300 to 182400 to ./data/medium.article.topic.csv
Current file size: 944963598 bytes
2025-04-10 12:30:49 - Processing chunk starting at row 182400, ending at row 182500
2025-04-10 12:31:20 - Written rows 182400 to 182500 to ./data/medium.article.topic.csv
Current file size: 945522006 bytes
2025-04-10 12:31:20 - Processing chunk starting at row 182500, ending at row 182600
2025-04-10 12:31:58 - Written rows 182500 to 182600 to ./data/medium.article.topic.csv
Current file size: 945996396 bytes
2025-04-10 12:31:58 - Processing chunk starting at row 182600, ending at row 182700
2025-04-10 12:32:31 - Written rows 182600 to 182700 to ./data/medium.article.topic.csv
Current file size: 946452656 bytes
2025-04-10 12:32:31 - Processing chunk starting at row

In [2]:
# check the new dataframe contents
new_df.head()

NameError: name 'new_df' is not defined

In [1]:
# Check the contents of the directory after writing
!ls -lh ./data/

total 1.9G
-rw-r--r-- 1 jupyter jupyter 949M Apr 10 09:24 medium.article.topic.csv
-rw-r--r-- 1 jupyter jupyter 995M Apr  9 12:11 medium_articles.csv


#### Next, train BERT (a much less expensive model) to perform the Topic identification.

##### set up science kit

In [76]:
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 34.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 56.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 45.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 142.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 34.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 107.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 106.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 107.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 66.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 77.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
from google.cloud import aiplatform, bigquery
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

##### load new dataframe

In [5]:
bert_df = pd.read_csv('./data/medium.article.topic.csv')

##### check dataframe format

In [6]:
# Display first few rows
bert_df.head()

,text,topic
0,Photo by Josh Riemer on Unsplash\n\nMerry Chri...,personal development
1,Your Brain On Coronavirus\n\nA guide to the cu...,medicine
2,Mind Your Nose\n\nHow smell training can chang...,science
3,Passionate about the synergy between science a...,technology
4,"You’ve heard of him, haven’t you? Phineas Gage...",science


##### split out new training, validation, and test datasets

In [7]:
from sklearn.model_selection import train_test_split

# Split into training and temp (validation + test) set (70% train, 30% temp)
train, temp = train_test_split(bert_df, test_size=0.3, random_state=42)

# Split the temp set into validation and test set (50% validation, 50% test of the 30%)
validate, test = train_test_split(temp, test_size=0.5, random_state=42)

# Print counts of rows in each set
print(f"Training Set Size: {train.shape[0]}")
print(f"Validation Set Size: {validate.shape[0]}")
print(f"Test Set Size: {test.shape[0]}")

Training Set Size: 134657
Validation Set Size: 28855
Test Set Size: 28856


##### set up training prereqs

In [80]:
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 79.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.17.2
    Uninstalling huggingface-hub-0.17.2:
      Successfully uninstalled huggingface-hub-0.17.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.14.5 requires fsspec[http]<2023.9.0,>=2023.1.0, but you have fsspec 2025.3.0 which is incompatible.


##### Create and train model

In [16]:
import datetime
import shutil

import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from google.cloud import aiplatform
from official.nlp import optimization  # to create AdamW optmizer

tf.get_logger().setLevel("ERROR")

In [17]:
bert_df.head()

,text,topic
0,Photo by Josh Riemer on Unsplash\n\nMerry Chri...,personal development
1,Your Brain On Coronavirus\n\nA guide to the cu...,medicine
2,Mind Your Nose\n\nHow smell training can chang...,science
3,Passionate about the synergy between science a...,technology
4,"You’ve heard of him, haven’t you? Phineas Gage...",science


In [20]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

BATCH_SIZE = 32

# Fit label encoder on all labels
label_encoder = LabelEncoder()
label_encoder.fit(pd.concat([train["topic"], validate["topic"], test["topic"]]))

# Encode the labels
train_labels = label_encoder.transform(train["topic"])
val_labels   = label_encoder.transform(validate["topic"])
test_labels  = label_encoder.transform(test["topic"])

# Convert DataFrames to tf.data.Dataset
raw_train_ds = tf.data.Dataset.from_tensor_slices((train["text"].values, train_labels))
val_ds       = tf.data.Dataset.from_tensor_slices((validate["text"].values, val_labels))
test_ds      = tf.data.Dataset.from_tensor_slices((test["text"].values, test_labels))

# Shuffle and batch the training set
raw_train_ds = raw_train_ds.shuffle(buffer_size=len(train)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_ds       = val_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_ds      = test_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Print dataset sizes (samples, not batches)
print(f"Train samples: {train.shape[0]}")
print(f"Validation samples: {validate.shape[0]}")
print(f"Test samples: {test.shape[0]}")

Train samples: 134657
Validation samples: 28855
Test samples: 28856


##### Check splits

In [24]:
class_names = label_encoder.classes_

for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(3):
        print(f"Text  : {text_batch.numpy()[i].decode('utf-8')}")
        label = label_batch.numpy()[i]
        print(f"Label : {label} ({class_names[label]})")
        print("---")

Text  : PlayOjo is a great site for new players in the UK. It is licensed in Malta and the UK and has the same selection of games as its online counterpart. Players in the UK will also find that the casino is very user-friendly. The interface is colorful and easy to use. You can also access a number of exclusive competitions and guides to games at PlayOjo. The gaming industry is booming in Malta, and the casino is doing very well in the country.

You can contact the support team via live chat or email. Live chat is usually quick, but you should check the FAQ first to see if there is an answer to your question. You can also check their blog or FAQ section for any related questions. Once you have registered with the casino, you can begin playing. The site will even give you a free trial of the games. Besides that, the customer service team is very friendly and will be happy to assist you.

The support team at Play OJO login UK is friendly and will answer any questions you have about the 

##### Choose initial bert model

In [25]:
# defining the URL of the smallBERT model to use
tfhub_handle_encoder = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"

# defining the corresponding preprocessing model for the BERT model above
tfhub_handle_preprocess = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"

print(f"BERT model selected           : {tfhub_handle_encoder}")
print(f"Preprocess model auto-selected: {tfhub_handle_preprocess}")

BERT model selected           : https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


##### Load it into Keras

In [26]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

##### Initial preprocessor validation

In [27]:
text_test = ["this is such an amazing movie!"]
text_preprocessed = bert_preprocess_model(text_test)

# This print box will help you inspect the keys in the pre-processed dictionary
print(f"Keys       : {list(text_preprocessed.keys())}")

# 1. input_word_ids is the ids for the words in the tokenized sentence
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')

# 2. input_mask is the tokens which we are masking (masked language model)
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')

# 3. input_type_ids is the sentence id of the input sentence.
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

Keys       : ['input_type_ids', 'input_word_ids', 'input_mask']
Shape      : (1, 128)
Word Ids   : [ 101 2023 2003 2107 2019 6429 3185  999  102    0    0    0]
Input Mask : [1 1 1 1 1 1 1 1 1 0 0 0]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]


#### Optimize frame size

In [30]:
from transformers import BertTokenizer
import numpy as np

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def get_token_lengths(text_batches):
    # Flattening the batch into a list of tokenized lengths
    return [len(tokenizer.tokenize(text.decode('utf-8'))) for text in text_batches]

# Initialize the lists to collect token lengths
token_lengths = []

# Iterate through a single batch of data in raw_train_ds
for text_batch, label_batch in raw_train_ds.take(1):  # Assuming raw_train_ds is the dataset you've prepared
    # Get token lengths for this batch of texts
    token_lengths.extend(get_token_lengths(text_batch))

# Analyze the token lengths
print("Token count stats:")
print(f"Mean      : {np.mean(token_lengths):.1f}")
print(f"Median    : {np.median(token_lengths):.1f}")
print(f"95th perc : {np.percentile(token_lengths, 95):.0f}")
print(f"Max       : {np.max(token_lengths)}")

# Print dataset sizes
print(f"Train samples: {train.shape[0]}")
print(f"Validation samples: {validate.shape[0]}")
print(f"Test samples: {test.shape[0]}")

# Sanity check by printing a few text examples from the batch
class_names = label_encoder.classes_

for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(3):
        print(f"Text  : {text_batch.numpy()[i].decode('utf-8')}")
        label = label_batch.numpy()[i]
        print(f"Label : {label} ({class_names[label]})")
        print("---")

RuntimeError: Only a single TORCH_LIBRARY can be used to register the namespace prims; please put all of your definitions in a single TORCH_LIBRARY block.  If you were trying to specify implementations, consider using TORCH_LIBRARY_IMPL (which can be duplicated).  If you really intended to define operators for a single namespace in a distributed way, you can use TORCH_LIBRARY_FRAGMENT to explicitly indicate this.  Previous registration of TORCH_LIBRARY was registered at /dev/null:241; latest registration was registered at /dev/null:241

##### Step 5: Main function to run the workflow

##### kubeflow model with parallel training and comparison testing, and hypertuning using k8s

#### Sport and Score Extractor

input: article body  
output: identified sport, score summary

In [ ]:

# Import necessary modules from pydantic
from pydantic import BaseModel, Extra
from typing import Dict, Any
import pandas as pd  # Assuming you're using pandas for the test dataframe

# Define the Schema class
class Schema(BaseModel):
    type: str
    properties: Dict[str, Any]
    required: list

# Define the FunctionDeclarationWithExtra class
class FunctionDeclarationWithExtra(BaseModel):
    name: str
    description: str
    parameters: Schema
    # Allow extra fields (like PROMPT)
    class Config:
        extra = Extra.allow

# Define the function declaration for score extraction
score_extraction_func = FunctionDeclarationWithExtra(
    name="score_extraction",
    description="Extract the scores from the article if feasible",
    parameters=Schema(
        type="OBJECT",
        properties={
            "article_text": {  # Define the expected 'article_text' input
                "type": "STRING",
                "description": "The content of the article to extract scores."
            },
            "score_available": {
                "type": "BOOLEAN",
                "description": "Indicates if scores are available in the article."
            },
            "team1_score": {
                "type": "INTEGER",
                "description": "Score for team 1 if available."
            },
            "team2_score": {
                "type": "INTEGER",
                "description": "Score for team 2 if available."
            },
            "team1_name": {
                "type": "STRING",
                "description": "Name of team 1 if mentioned."
            },
            "team2_name": {
                "type": "STRING",
                "description": "Name of team 2 if mentioned."
            },
        },
        required=["article_text", "score_available"],
    ),
    PROMPT="""
    Given the following article, extract the score if feasible:

    {article_text}

    Provide the output in a structured JSON format:
    {{
        "score_available": true/false,
        "team1_name": "name_of_team1",
        "team1_score": score_of_team1,
        "team2_name": "name_of_team2",
        "team2_score": score_of_team2
    }}
    """
)

def extract_scores_from_article(article_text: str):
    # Dynamically set the prompt with the actual article text
    formatted_prompt = score_extraction_func.PROMPT.format(article_text=article_text)

    # Example client and content generation logic (ensure the client and model are correctly defined)
    response = client.models.generate_content(
        model=MODEL,
        contents=formatted_prompt,
        config=GenerateContentConfig(
            response_mime_type="application/json",
            response_schema={
                "type": "OBJECT",
                "properties": {
                    "score_available": {
                        "type": "BOOLEAN",
                    },
                    "team1_name": {
                        "type": "STRING",
                    },
                    "team1_score": {
                        "type": "INTEGER",
                    },
                    "team2_name": {
                        "type": "STRING",
                    },
                    "team2_score": {
                        "type": "INTEGER",
                    }
                },
                "required": ["score_available"],
            },
        ),
    )

    # The response contains the scores extracted from the article
    scores_extraction_result = json.loads(response.json())  # Parse the JSON response

    return scores_extraction_result

# Loop through the test dataframe and extract scores
for idx, row in test.head(2000).iterrows():  # Adjust the dataframe as necessary
    article_text = row["text"]
    extracted_scores = extract_scores_from_article(article_text)
    actual_tags = row["tags"]  # Get the tags for the current row

    if extracted_scores.get("score_available"):
        print(f"Extracted Scores: {extracted_scores}\t\tActual tags: {actual_tags}")

#### Sport Statistics Summarizer

input: article body  
output: statistics summary

In [ ]:

# Import necessary modules from pydantic
from pydantic import BaseModel, Extra
from typing import Dict, Any
import pandas as pd  # Assuming you're using pandas for the test dataframe
import json

# Define the Schema class
class Schema(BaseModel):
    type: str
    properties: Dict[str, Any]
    required: list

# Define the FunctionDeclarationWithExtra class
class FunctionDeclarationWithExtra(BaseModel):
    name: str
    description: str
    parameters: Schema
    # Allow extra fields (like PROMPT)
    class Config:
        extra = Extra.allow

# Define the function declaration for sport mention and person summary extraction
sport_person_extraction_func = FunctionDeclarationWithExtra(
    name="sport_person_extraction",
    description="Extract mentions of sports and summarize information related to individuals mentioned in the article if feasible",
    parameters=Schema(
        type="OBJECT",
        properties={
            "article_text": {  # Define the expected 'article_text' input
                "type": "STRING",
                "description": "The content of the article to extract sport mentions and person summaries."
            },
            "sport_mentioned": {
                "type": "BOOLEAN",
                "description": "Indicates if any sports are mentioned in the article."
            },
            "person_name": {
                "type": "STRING",
                "description": "Name of the person mentioned in relation to the sport."
            },
            "summary": {
                "type": "STRING",
                "description": "Summary of what is said about the person."
            },
        },
        required=["article_text", "sport_mentioned"],
    ),
    PROMPT="""
    Given the following article, identify if any sports are mentioned. If there are mentions of sports, find any names attached to the article and provide a summary about what the article is saying about that person.

    {article_text}

    Provide the output in a structured JSON format:
    {{
        "sport_mentioned": true/false,
        "person_name": "name_of_person_mentioned",
        "summary": "summary_about_person"
    }}
    """
)

def extract_sport_person_summary_from_article(article_text: str):
    # Dynamically set the prompt with the actual article text
    formatted_prompt = sport_person_extraction_func.PROMPT.format(article_text=article_text)

    # Example client and content generation logic (ensure the client and model are correctly defined)
    response = client.models.generate_content(
        model=MODEL,  # Make sure to replace with the actual model name you are using
        contents=formatted_prompt,
        config=GenerateContentConfig(
            response_mime_type="application/json",
            response_schema={
                "type": "OBJECT",
                "properties": {
                    "sport_mentioned": {
                        "type": "BOOLEAN",
                    },
                    "person_name": {
                        "type": "STRING",
                    },
                    "summary": {
                        "type": "STRING",
                    }
                },
                "required": ["sport_mentioned"],
            },
        ),
    )

    # Parse the JSON response
    response_data = json.loads(response.json())
    sport_person_extraction_result = response_data.get('parsed', {})

    return sport_person_extraction_result

# Loop through the test dataframe and extract sport mentions and person summaries
for idx, row in test.head(2000).iterrows():
    article_text = row["text"]
    extracted_info = extract_sport_person_summary_from_article(article_text)
    actual_tags = row["tags"]  # Get the tags for the current row

    if extracted_info.get("sport_mentioned") == True:
        print(f"Extracted Info: {extracted_info}\t\tActual tags: {actual_tags}")

### API Deployment (Input/Output)

#### Stand up Webapp endpoint

##### Dockerize instance with Python scripting and the weighted BERT model

##### Create Support Pipeline

##### Stand up Vertex Instance (the actual endpoint)

## Reference Labs

- Gemini Function Calling
- Gemini Prompt Engineering
- AutoML for Text Classification - Vertex
- KFP Walkthrogh - Vertex Containerization - Training and Deployment Pipelines